<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/learnelixir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

elixir を齧る。かじる。

今のイメージ erlang 上で、erlang は 並行処理のためのシステムで、その erlang 上で理想的な言語を作ろうとしたら、ruby + clojure みたいな言語になった。

Dave Thomas と まつもとゆきひろ が勧めているのだからいい言語なのだろう。


* https://magazine.rubyist.net/articles/0054/0054-ElixirBook.html
* https://elixirschool.com/ja/lessons/basics/basics/

In [11]:
%%capture
!sudo apt install elixir

In [19]:
!elixir -e 'IO.puts 3 + 3'

6
